Se generará el modelo específico para Amborella trichopoda.<br/>
Primero se realiza la instalación de paquetes.

In [1]:
# Install a pip package in the current Jupyter kernel
import sys
!{sys.executable} -m pip install numpy

In [2]:
# Install a pip package in the current Jupyter kernel
import sys
!{sys.executable} -m pip install biopython

In [3]:
# Install a pip package in the current Jupyter kernel
import sys
!{sys.executable} -m pip install scipy

In [4]:
# Install a pip package in the current Jupyter kernel
import sys
!{sys.executable} -m pip install scikit-learn

In [5]:
# Install a pip package in the current Jupyter kernel
import sys
!{sys.executable} -m pip install configparser

In [6]:
# Install a pip package in the current Jupyter kernel
import sys
!{sys.executable} -m pip install matplotlib

Generación de inputs en archivos fasta.

In [7]:
# Install a pip package in the current Jupyter kernel
import sys
!{sys.executable} -m pip install myloginpath

In [8]:
%%writefile ./libs/util_bd.py
import mysql.connector
import myloginpath
import pandas as pd

def resultados_query(query):
    conf = myloginpath.parse('tesis2')
    conn = mysql.connector.connect(**conf, db="tesis2")
    cursor = conn.cursor()
    cursor.execute(query)
    resultado = cursor.fetchall()
    conn.close()
    return resultado

def ejecutar_query(query):
    conf = myloginpath.parse('tesis2')
    conn = mysql.connector.connect(**conf, db="tesis2")
    cursor = conn.cursor()
    cursor.execute(query)
    conn.close()

def mostrar_resultado_query(query):
    conf = myloginpath.parse('tesis2')
    conn = mysql.connector.connect(**conf, db="tesis2")
    df = pd.read_sql_query(query, conn)
    display(df)
    conn.close()

Overwriting ./libs/util_bd.py


In [9]:
%%writefile ./libs/util_fasta.py
import os

def generar_fasta(secuencias, archivo):
    t_tamanio = 80
    f = open(archivo ,"w+")
    for transcrito in secuencias:
        f.write(">%s\n" % (transcrito[0].strip().upper()))
        seq = transcrito[1]
        t_partes = [seq[i:i+t_tamanio] for i in range(0, len(seq), t_tamanio)]
        for t_parte in t_partes:
            f.write("%s\n" % (t_parte))
    f.close()

def leer_fasta(archivo):
    transcritos = {}
    cod_secuencia = ""
    secuencia = ""
    f = open(archivo, "r")
    for linea in f:
        if linea.startswith(">"):
            if secuencia != "":
                transcritos[cod_secuencia] = secuencia
                secuencia = ""
            cod_secuencia = linea.rstrip("\n").lstrip(">").strip().upper()
        else:
            secuencia += linea.rstrip("\n")
    if secuencia != "":
        transcritos[cod_secuencia] = secuencia
        secuencia = ""
    f.close()
    return transcritos

Overwriting ./libs/util_fasta.py


In [10]:
%%time
import sys
sys.path.append('./libs')
import util_bd, util_fasta

print("Iniciando proceso...")
id_especie = "2"

# lncRNA
print("Generando fasta lncRNA...") 
query = "SELECT cod_secuencia, secuencia FROM secuencias WHERE flg_pct = 0 AND flg_seleccionado = 1 AND id_especie = " + id_especie
secuencias = util_bd.resultados_query(query)
util_fasta.generar_fasta(secuencias, "./data/Especie" + id_especie + ".lncRNA.fasta")

# PCT
print("Generando fasta PCT...") 
query = "SELECT cod_secuencia, secuencia FROM secuencias WHERE flg_pct = 1 AND flg_seleccionado = 1 AND id_especie = " + id_especie
secuencias = util_bd.resultados_query(query)
util_fasta.generar_fasta(secuencias, "./data/Especie" + id_especie + ".PCT.fasta")

# CDS
print("Generando fasta CDS...") 
query = "SELECT f.cod_secuencia, cds.coding FROM secuencias_CDS cds JOIN secuencias_features f ON cds.id_especie = f.id_especie AND cds.cod_secuencia = f.cod_secuencia WHERE f.flg_pct = 1 AND f.flg_seleccionado = 1 AND f.id_especie = " + id_especie
secuencias = util_bd.resultados_query(query)
util_fasta.generar_fasta(secuencias, "./data/Especie" + id_especie + ".CDS.fasta")

print("Proceso terminado...")

Iniciando proceso...
Generando fasta lncRNA...
Generando fasta PCT...
Generando fasta CDS...
Proceso terminado...
CPU times: user 1.61 s, sys: 188 ms, total: 1.8 s
Wall time: 29.4 s


Generación de las características.

In [11]:
%%writefile ./libs/util_caracteristicas.py
import os
import util_bd, util_fasta
from Bio.SeqUtils import GC
import csv

def generar_modelo_CPAT(identificador, codigos_lncRNA, codigos_PCT):
    archivos = _rutas_archivos(identificador)
    if os.path.isdir(archivos["cpat"]["directorio_base"]):
        return
    _generar_directorios_cpat(archivos)
    _generar_data_cpat(archivos, codigos_lncRNA, codigos_PCT)
    _generar_modelo_cpat(archivos)

def generar_caracteristicas(identificador, transcritos):
    archivos = _rutas_archivos(identificador)
    if not os.path.isdir(archivos["cpat"]["directorio_base"]):
        raise Exception("No se encontró la carpeta del modelo CPAT {}, probablemente aún no ha generado este modelo. Ruta buscada: {}".format(identificador, archivos["cpat"]["directorio_base"]))
    _generar_transcritos_fasta(archivos, transcritos)
    _generar_caracteristicas_cpat(archivos, archivos["transcritos_fasta"])
    _generar_caracteristicas_diamond(archivos, archivos["transcritos_fasta"])
    return _generar_caracteristicas(archivos, transcritos)
    
def _rutas_archivos(identificador):
    archivos = {}
    archivos["cpat"] = { "directorio_base" : "./CPAT/{}".format(identificador) }
    archivos["cpat"]["data"] = { "directorio" : "{}/data".format(archivos["cpat"]["directorio_base"]) }
    archivos["cpat"]["data"]["lncRNA"] = "{}/lncRNA.fasta".format(archivos["cpat"]["data"]["directorio"])
    archivos["cpat"]["data"]["PCT"] = "{}/PCT.fasta".format(archivos["cpat"]["data"]["directorio"])
    archivos["cpat"]["data"]["CDS"] = "{}/CDS.fasta".format(archivos["cpat"]["data"]["directorio"])
    archivos["cpat"]["modelo"] = { "directorio" : "{}/modelo".format(archivos["cpat"]["directorio_base"]) }
    archivos["cpat"]["modelo"]["hexamer"] = "{}/hexamer.tsv".format(archivos["cpat"]["modelo"]["directorio"])
    archivos["cpat"]["modelo"]["prefijo_logit"] = "{}/{}".format(archivos["cpat"]["modelo"]["directorio"], identificador)
    archivos["cpat"]["modelo"]["logit"] = "{}.logit.RData".format(archivos["cpat"]["modelo"]["prefijo_logit"])
    archivos["cpat"]["modelo"]["prefijo_cpat"] = "{}/{}".format(archivos["cpat"]["modelo"]["directorio"], identificador)
    archivos["cpat"]["salida"] = "{}.dat".format(archivos["cpat"]["modelo"]["prefijo_cpat"])
    archivos["cpat"]["scripts"] = {
        "script_hexamer" : "~/anaconda3/bin/make_hexamer_tab.py",
        "script_logit" : "~/anaconda3/bin/make_logitModel.py",
        "script_cpat" : "~/anaconda3/bin/cpat.py"
    }
    archivos["diamond"] = { "directorio_base" : "./Diamond" }
    archivos["diamond"]["bd"] = "{}/uniprot-viridiplantae-reviewed.dmnd".format(archivos["diamond"]["directorio_base"])
    archivos["diamond"]["script"] = "~/anaconda3/bin/diamond"
    archivos["diamond"]["salida"] = "{}/{}.tsv".format(archivos["diamond"]["directorio_base"], identificador)
    archivos["transcritos_fasta"] = "./data/{}.fasta".format(identificador)
    return archivos
    
def _generar_directorios_cpat(archivos):
    os.mkdir(archivos["cpat"]["directorio_base"])
    os.mkdir(archivos["cpat"]["data"]["directorio"])
    os.mkdir(archivos["cpat"]["modelo"]["directorio"])

def _generar_data_cpat(archivos, codigos_lncRNA, codigos_PCT):
    query = "SELECT cod_secuencia, secuencia FROM secuencias WHERE cod_secuencia IN ('{}')".format("', '".join(codigos_lncRNA))
    secuencias = util_bd.resultados_query(query)
    util_fasta.generar_fasta(secuencias, archivos["cpat"]["data"]["lncRNA"])
    query = "SELECT cod_secuencia, secuencia FROM secuencias WHERE cod_secuencia IN ('{}')".format("', '".join(codigos_PCT))
    secuencias = util_bd.resultados_query(query)
    util_fasta.generar_fasta(secuencias, archivos["cpat"]["data"]["PCT"])
    query = "SELECT cod_secuencia, coding FROM secuencias_CDS WHERE cod_secuencia IN ('{}')".format("', '".join(codigos_PCT))
    secuencias = util_bd.resultados_query(query)
    util_fasta.generar_fasta(secuencias, archivos["cpat"]["data"]["CDS"])

def _generar_modelo_cpat(archivos):
    _generar_hexamer_cpat(archivos)
    _generar_logit_cpat(archivos)
    
def _generar_hexamer_cpat(archivos):
    script = archivos["cpat"]["scripts"]["script_hexamer"]
    fasta_cds = "'" + archivos["cpat"]["data"]["CDS"] + "'" 
    fasta_lncRNA = "'" + archivos["cpat"]["data"]["lncRNA"] + "'"
    salida = "'" + archivos["cpat"]["modelo"]["hexamer"] + "'"
    comando = "{} -c {} -n {} > {}".format(script, fasta_cds, fasta_lncRNA, salida)
    os.system(comando)
    
def _generar_logit_cpat(archivos):
    script = archivos["cpat"]["scripts"]["script_logit"]
    hexamer = "'" + archivos["cpat"]["modelo"]["hexamer"] + "'"
    fasta_pct = "'" + archivos["cpat"]["data"]["PCT"] + "'" 
    fasta_lncRNA = "'" + archivos["cpat"]["data"]["lncRNA"] + "'"
    salida = "'" + archivos["cpat"]["modelo"]["prefijo_logit"] + "'"
    comando = "{} -x {} -c {} -n {} -o {}".format(script, hexamer, fasta_pct, fasta_lncRNA, salida)
    os.system(comando)

def _generar_transcritos_fasta(archivos, transcritos):
    transcritos_array = transcritos.items()
    util_fasta.generar_fasta(transcritos_array, archivos["transcritos_fasta"])
    
def _generar_caracteristicas_cpat(archivos, transcritos_fasta):
    script = archivos["cpat"]["scripts"]["script_cpat"]
    logit = "'" + archivos["cpat"]["modelo"]["logit"] + "'"
    hexamer = "'" + archivos["cpat"]["modelo"]["hexamer"] + "'"
    salida = "'" + archivos["cpat"]["modelo"]["prefijo_cpat"] + "'"
    comando = "{} -g {} -d {} -x {} -o {}".format(script, transcritos_fasta, logit, hexamer, salida)
    os.system(comando)

def _generar_caracteristicas_diamond(archivos, transcritos_fasta):
    script = archivos["diamond"]["script"]
    diamond_bd = "'" + archivos["diamond"]["bd"] + "'"
    salida = "'" + archivos["diamond"]["salida"] + "'"
    comando = "{} blastx -d {} -q {} -o {} -k 5 --gapopen 11 --gapextend 1 --more-sensitive -f 6 qseqid pident length qframe qstart qend sstart send evalue bitscore".format(script, diamond_bd, transcritos_fasta, salida)
    os.system(comando)

def _generar_caracteristicas(archivos, transcritos):
    transcript_dict = {}
    for k in transcritos.keys():
        transcript_dict[k.strip().upper()] = {
            "length" : len(transcritos[k]),
            "gc" : GC(transcritos[k]),
            "orf_length" : 0,
            "orf_coverage" : float(0),
            "hexamer_score" : float(0),
            "fickett_score" : float(0),
            "identity" : float(0),
            "align_length" : float(0),
            "align_perc_len" : float(0),
            "align_perc_orf" : float(0)
        }
    
    with open(archivos["cpat"]["salida"], "r") as f:
        cpat_reader = csv.reader(f, delimiter=("\t"))
        for row in cpat_reader:
            cod_secuencia = row[0]
            transcript_dict[cod_secuencia]["orf_length"] = float(row[2])
            transcript_dict[cod_secuencia]["orf_coverage"] = float(row[2])/float(transcript_dict[cod_secuencia]["length"])
            transcript_dict[cod_secuencia]["fickett_score"] = float(row[3])
            transcript_dict[cod_secuencia]["hexamer_score"] = float(row[4])
    
    #adaptado de https://github.com/gbgolding/crema/blob/master/bin/featuresetup_module.py
    with open(archivos["diamond"]["salida"], "r") as f:
        tab_reader = csv.reader(f, delimiter=("\t"))
        line_1 = next(tab_reader)
        first = line_1[0].upper()
        score = [float(line_1[9])]
        with_len = [[first, float(line_1[1]), float(line_1[2]), float(line_1[3]), float(line_1[9])]] # name identity length frame score
        for row in tab_reader:
            if row[0].upper() == first:
                score.append(float(row[9]))
                with_len.append([row[0].upper(), float(row[1]), float(row[2]), float(row[3]), float(row[9])])
            else:
                transcript_dict[first]["identity"] = float(0)
                transcript_dict[first]["align_length"] = float(0)
                max_value = max(score)
                max_index = score.index(max_value)
                max_len_ident = with_len[max_index]
                if max_len_ident[3] > 0:
                    transcript_dict[first]["identity"] = float(max_len_ident[1])
                    transcript_dict[first]["align_length"] = float(max_len_ident[2])
                    transcript_dict[first]["align_perc_len"] = float(transcript_dict[first]["align_length"]/transcript_dict[first]["length"])
                    transcript_dict[first]["align_perc_orf"] = (0 if transcript_dict[first]["orf_length"] == 0 else float(transcript_dict[first]["align_length"]/transcript_dict[first]["orf_length"]))
                score = [float(row[9])]
                first = row[0].upper()
                with_len = [[first, float(row[1]), float(row[2]), float(row[3]), float(row[9])]]
        transcript_dict[first]["identity"] = float(0)
        transcript_dict[first]["align_length"] = float(0)
        max_value = max(score)
        max_index = score.index(max_value)
        max_len_ident = with_len[max_index]
        if max_len_ident[3] > 0:
            transcript_dict[first]["identity"] = float(max_len_ident[1])
            transcript_dict[first]["align_length"] = float(max_len_ident[2])
    #fin de código adaptado de https://github.com/gbgolding/crema/blob/master/bin/featuresetup_module.py
    
    return transcript_dict

Overwriting ./libs/util_caracteristicas.py


Prueba de generación de características

In [ ]:
%%time
import sys
sys.path.append('./libs')
import util_caracteristicas, util_fasta
import pandas as pd
import os, shutil

print("Iniciando proceso...") 

identificador = "Especie2"

if os.path.isdir("./CPAT/" + identificador):
     shutil.rmtree("./CPAT/" + identificador)

columnas = ["length", "gc", "orf_length", "orf_coverage", "hexamer_score", "fickett_score", "identity", "align_length", "align_perc_len", "align_perc_orf"]

print("Leyendo fasta lncRNA...")
codigos_lncRNA = util_fasta.leer_fasta("./data/" + identificador + ".lncRNA.fasta")
print("Leyendo fasta PCT...")
codigos_PCT = util_fasta.leer_fasta("./data/" + identificador + ".PCT.fasta")
print("Leyendo fasta CDS...")
codigos_CDS = util_fasta.leer_fasta("./data/" + identificador + ".CDS.fasta")
print("Generando modelo CPAT...")
util_caracteristicas.generar_modelo_CPAT(identificador, codigos_lncRNA.keys(), codigos_PCT.keys())
print("Prueba lncRNA")
resultado = util_caracteristicas.generar_caracteristicas(identificador, codigos_lncRNA)
display(pd.DataFrame.from_dict(resultado, orient='index', columns=columnas).head())
display(pd.DataFrame.from_dict(resultado, orient='index', columns=columnas).mean(axis=0))
print("Prueba PCT")
resultado = util_caracteristicas.generar_caracteristicas(identificador, codigos_PCT)
display(pd.DataFrame.from_dict(resultado, orient='index', columns=columnas).head())
display(pd.DataFrame.from_dict(resultado, orient='index', columns=columnas).mean(axis=0))
print("Prueba CDS")
resultado = util_caracteristicas.generar_caracteristicas(identificador, codigos_CDS)
display(pd.DataFrame.from_dict(resultado, orient='index', columns=columnas).head())
display(pd.DataFrame.from_dict(resultado, orient='index', columns=columnas).mean(axis=0))

print("Proceso finalizado...")

Iniciando proceso...
Leyendo fasta lncRNA...
Leyendo fasta PCT...
Leyendo fasta CDS...
Generando modelo CPAT...
Prueba lncRNA


,length,gc,orf_length,orf_coverage,hexamer_score,fickett_score,identity,align_length,align_perc_len,align_perc_orf
LCL|ATRICHOPODA_EVM_27.MODEL.AMTR_V1.0_SCAFFOLD00001.100,351,47.293447,351.0,1.0,0.090494,0.9142,0.0,0.0,0.000000,0.000000
LCL|ATRICHOPODA_EVM_27.MODEL.AMTR_V1.0_SCAFFOLD00001.120,285,49.122807,285.0,1.0,0.119809,0.9125,40.4,104.0,0.364912,0.364912
LCL|ATRICHOPODA_EVM_27.MODEL.AMTR_V1.0_SCAFFOLD00001.133,231,38.961039,231.0,1.0,0.055321,0.6864,0.0,0.0,0.000000,0.000000
LCL|ATRICHOPODA_EVM_27.MODEL.AMTR_V1.0_SCAFFOLD00001.142,279,45.519713,279.0,1.0,0.037444,0.4407,0.0,0.0,0.000000,0.000000
LCL|ATRICHOPODA_EVM_27.MODEL.AMTR_V1.0_SCAFFOLD00001.181,354,48.022599,354.0,1.0,0.125634,0.8414,0.0,0.0,0.000000,0.000000


length            288.955323
gc                 48.080142
orf_length        277.168489
orf_coverage        0.965150
hexamer_score       0.058192
fickett_score       0.880493
identity           10.783389
align_length       15.074620
align_perc_len      0.051645
align_perc_orf      0.057322
dtype: float64

Prueba PCT
